In [129]:
import glob
# This file should run basically insta fast. Bad coding for sure 
genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = "03" # Which month and days result you want to visualize in table
month = 11
repetitionValue = 2
repetitionName = "udrlBC2CVTest"
repetitionString = f"{repetitionValue}{repetitionName}"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionString}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.5
print(len(savedResults))
print(len(savedResults)/9)

9
1.0


In [130]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                if ares > 0.98:
                    ares = 0.99
                akernel = aResults2[2]
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                  
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
topThreeAllSubjects = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    topThreeThisSubject = []
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()

            
            
            if len(topThreeThisSubject) < 2 or topThreeThisSubject[0][0] < res[1]:

                if len(topThreeThisSubject) == 2:
                    topThreeThisSubject.reverse()
                    topThreeThisSubject.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topThreeThisSubject.append([res[1], subNr, dataF, kernel, cVal])
                topThreeThisSubject.sort()
    if len(topThreeThisSubject)>0:
        topThreeAllSubjects.append(topThreeThisSubject)
# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)




(1, 84)
(10, 84)


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_7276\1128782281.py:188: RuntimeWarning: invalid value encountered in divide
  comboAverage = comboAverage / nrOfSubjects


,0,1,2,3,4,5,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,0.625,0.6,0.725,0.775,0.8,0.775,0.6,0.725,0.625,0.55,None,None,0.675,0.625,0.625,0.75,0.775,0.75,0.625,0.65,0.725,0.6,0.65,0.675,0.625,0.7,0.75,0.625,0.625,0.65,0.725,0.725,0.65,0.675,0.75,0.725,0.65,0.7,0.675,0.675,0.725,0.7,0.65,0.65,0.725,0.7,0.675,0.725,0.625,0.75,0.65,0.6,None,None,0.675,0.6,0.65,0.6,0.725,0.625,0.575,None,None,0.625,0.65,0.7,0.625,0.7,0.7,0.7,0.8,0.75,0.675,0.575,0.7
2,0.791667,0.791667,0.875,0.75,0.791667,0.75,0.6875,0.6875,0.729167,0.645833,0.645833,0.666667,0.625,0.645833,0.583333,0.770833,0.75,0.833333,0.770833,0.791667,0.854167,0.666667,0.791667,0.875,0.8125,0.770833,0.791667,0.854167,0.75,0.8125,0.625,0.708333,0.666667,0.729167,0.75,0.729167,0.708333,0.75,0.6875,0.770833,0.791667,0.708333,0.770833,0.791667,0.729167,0.75,0.6875,0.6875,0.708333,0.6875,0.75,0.625,0.666667,0.666667,0.645833,0.645833,0.583333,0.729167,0.6875,0.75,0.604167,0.645833,0.708333,0.625,0.645833,0.583333,0.666667,0.75,0.75,0.666667,0.6875,0.708333,0.645833,0.708333,0.583333
3,0.555556,0.527778,0.694444,0.777778,0.694444,0.75,0.583333,0.722222,0.694444,0.666667,0.527778,0.583333,0.5,0.527778,0.5,0.694444,0.805556,0.666667,0.527778,None,0.666667,0.666667,0.555556,0.666667,0.583333,0.75,0.638889,0.666667,0.638889,0.555556,0.583333,0.583333,0.5,0.75,0.75,0.722222,0.777778,0.722222,0.75,0.638889,0.75,0.722222,0.694444,0.722222,0.694444,0.583333,0.666667,0.611111,0.611111,0.722222,0.694444,0.638889,0.5,None,0.5,0.527778,0.5,0.666667,0.75,0.722222,0.583333,0.555556,0.527778,None,0.527778,0.5,0.638889,0.638889,0.694444,0.583333,0.666667,0.666667,None,None,None
4,0.6875,0.6875,0.708333,0.791667,0.770833,0.8125,0.708333,0.6875,0.75,0.541667,None,0.5,0.729167,None,0.708333,0.770833,0.8125,0.75,0.8125,0.729167,0.708333,0.5,0.708333,0.666667,0.6875,0.708333,0.75,0.645833,0.604167,0.645833,0.75,0.520833,0.729167,0.791667,0.75,0.791667,0.708333,0.729167,0.729167,0.770833,0.6875,0.75,0.770833,0.5625,0.645833,0.708333,0.645833,0.708333,0.75,0.6875,0.75,0.5625,None,0.520833,0.708333,None,0.708333,0.625,0.6875,0.729167,0.520833,None,None,0.645833,None,0.6875,0.708333,0.666667,0.75,0.770833,0.604167,0.75,0.75,0.5,0.6875
5,0.645833,0.625,0.708333,0.770833,0.708333,0.708333,0.604167,0.6875,0.75,0.583333,0.541667,0.604167,0.666667,0.6875,0.666667,0.791667,0.770833,0.708333,0.645833,0.645833,0.666667,0.520833,0.6875,0.729167,0.625,0.6875,0.75,0.604167,0.708333,0.708333,0.666667,0.6875,0.708333,0.8125,0.708333,0.708333,0.8125,0.729167,0.708333,0.770833,0.770833,0.75,0.8125,0.6875,0.6875,0.708333,0.6875,0.645833,0.625,0.6875,0.708333,0.583333,0.520833,0.625,0.645833,0.666667,0.645833,0.6875,0.6875,0.708333,0.541667,0.583333,0.625,0.6875,0.666667,0.604167,0.645833,0.708333,0.729167,0.666667,0.708333,0.625,0.541667,0.6875,0.625
6,0.613636,0.681818,0.659091,0.727273,0.704545,0.75,0.704545,0.727273,0.727273,0.5,0.545455,0.545455,0.590909,0.568182,0.590909,0.727273,0.795455,0.75,0.727273,0.704545,0.681818,0.636364,0.727273,0.659091,0.772727,0.727273,0.704545,0.681818,0.704545,0.659091,0.659091,0.636364,0.636364,0.75,0.727273,0.681818,0.727273,0.704545,0.681818,0.681818,0.704545,0.727273,0.681818,0.681818,0.704545,0.636364,0.659091,0.704545,0.727273,0.727273,0.727273,0.545455,0.522727,0.522727,0.613636,0.590909,0.590909,0.727273,0.727273,0.704545,0.545455,None,0.5,0.613636,0.545455,0.590909,0.681818,0.659091,0.704545,0.704545,0.659091,0.681818,0.590909,0.568182,0.613636
7

In [131]:
print("Top ten highest accuracies")

topTen.reverse()
for top in topTen:
    print(
            f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
        )


Top ten highest accuracies
Accuracy 0.88 : subject 2, feature:gaussianDatacn3--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.88 : subject 2, feature:-dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.85 : subject 2, feature:dataGCVcn3--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.85 : subject 2, feature:dataGCV-BCcn3--dataFFTCV-BC, kernellinear, C:0.5
Accuracy 0.83 : subject 7, feature:-fftDataBC, kernelsigmoid, C:2.5
Accuracy 0.83 : subject 7, feature:-fftDataBC, kernellinear, C:0.5
Accuracy 0.83 : subject 2, feature:fftDataBC--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.81 : subject 4, feature:fftDataBC--dataFFTCV-BC, kernelsigmoid, C:2.5
Accuracy 0.81 : subject 4, feature:dataGCVcn3--dataFFTCV-BC, kernellinear, C:0.5
Accuracy 0.81 : subject 4, feature:-fftDataBC, kernelrbf, C:2.5


In [132]:
print("Top ten highest accuracies")
#del(topThreeThisSubject[0])
#len(topThreeAllSubjects[0])
for subTop in topThreeAllSubjects:
    subTop.reverse()
topThreeAllSubjects.sort(reverse=True)
for subTop in topThreeAllSubjects:
    print("")
    #subTop.reverse()
    for top in subTop:
        print(
                f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
            )

Top ten highest accuracies

Accuracy 0.88 : subject 2, feature:gaussianDatacn3--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.88 : subject 2, feature:-dataFFTCV-BC, kernelrbf, C:2.5

Accuracy 0.83 : subject 7, feature:-fftDataBC, kernelsigmoid, C:2.5
Accuracy 0.83 : subject 7, feature:-fftDataBC, kernellinear, C:0.5

Accuracy 0.81 : subject 5, feature:gaussianDatacn3--fftDataBC, kernellinear, C:0.5
Accuracy 0.81 : subject 5, feature:dataGCVcn3--fftDataBC, kernellinear, C:0.5

Accuracy 0.81 : subject 4, feature:fftDataBC--dataFFTCV-BC, kernelsigmoid, C:2.5
Accuracy 0.81 : subject 4, feature:-fftDataBC, kernelrbf, C:2.5

Accuracy 0.81 : subject 3, feature:fftDataBC--dataFFTCV-BC, kernelsigmoid, C:2.5
Accuracy 0.78 : subject 3, feature:-fftDataBC, kernellinear, C:0.5

Accuracy 0.8 : subject 1, feature:dataGCV2-BCcn3--dataGCVcn3BC, kernelsigmoid, C:2.5
Accuracy 0.8 : subject 1, feature:-fftDataBC, kernelsigmoid, C:2.5

Accuracy 0.8 : subject 6, feature:fftDataBC--dataFFTCV-BC, kernelsigmoid, C

In [133]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)

print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
print(headers[maxInd])
print(nrOfSubjects[maxInd])
print(list(nameDict.keys())[combinationList[maxInd][0]])
print(list(kernelDict.keys())[combinationList[maxInd][1]])
print(f"C: {combinationList[maxInd][2]}")


[0.         0.67644928 0.71638889 0.63808374 0.69270833 0.67611111
 0.66308354 0.65434272 0.65101351 0.66964286]


,0,1,2,3,4,5,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83
0,0.63454,0.660054,0.6953,0.756117,0.752666,0.756481,0.655256,0.701333,0.712598,0.586979,0.557146,0.58566,0.603551,0.596447,0.611364,0.743617,0.76539,0.728241,0.651024,0.704221,0.697206,0.596633,0.665222,0.684343,0.670581,0.717845,0.722419,0.675331,0.652974,0.662276,0.638047,0.652983,0.639689,0.734259,0.73266,0.72869,0.734357,0.730752,0.711869,0.709245,0.734301,0.721703,0.726066,0.680542,0.705906,0.656355,0.670455,0.665629,0.658432,0.70874,0.708432,0.586454,0.547045,0.602045,0.603788,0.593146,0.60928,0.672475,0.702104,0.706215,0.562973,0.566944,0.596991,0.609686,0.590819,0.609801,0.655542,0.679405,0.711462,0.654672,0.686658,0.690109,0.603463,0.600812,0.638258


Max average accuracy 0.7653900112233446
22
9.0
-dataGCV-BCcn3
sigmoid
C: 2.5


In [134]:
x = 0
for names in nameDict.keys():
    print(names)
    x +=1
    print(x)

-dataFFTCV-BC
1
-fftDataBC
2
-dataGCVcn3
3
-gaussianDatacn3
4
-dataGCVcn3BC
5
-dataGCV-BCcn3
6
-dataGCV2-BCcn3
7
fftDataBC--dataFFTCV-BC
8
dataGCVcn3--dataFFTCV-BC
9
gaussianDatacn3--dataFFTCV-BC
10
dataGCVcn3BC--dataFFTCV-BC
11
dataGCV-BCcn3--dataFFTCV-BC
12
dataGCV2-BCcn3--dataFFTCV-BC
13
dataGCVcn3--fftDataBC
14
gaussianDatacn3--fftDataBC
15
dataGCVcn3BC--fftDataBC
16
dataGCV-BCcn3--fftDataBC
17
dataGCV2-BCcn3--fftDataBC
18
gaussianDatacn3--dataGCVcn3
19
dataGCVcn3BC--dataGCVcn3
20
dataGCV-BCcn3--dataGCVcn3
21
dataGCV2-BCcn3--dataGCVcn3
22
dataGCVcn3BC--gaussianDatacn3
23
dataGCV-BCcn3--gaussianDatacn3
24
dataGCV2-BCcn3--gaussianDatacn3
25
dataGCV-BCcn3--dataGCVcn3BC
26
dataGCV2-BCcn3--dataGCVcn3BC
27
dataGCV2-BCcn3--dataGCV-BCcn3
28


In [135]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,0,1,2,3,4,5,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83
0,5.710859,5.280429,6.257702,6.805051,6.77399,6.808333,5.242045,6.311995,6.413384,4.695833,2.785732,4.099621,4.828409,4.175126,4.890909,6.692551,6.88851,6.554167,5.859217,4.929545,5.577652,5.369697,5.986995,6.159091,6.035227,6.460606,6.501768,5.402652,5.876768,5.96048,5.742424,5.223864,5.757197,6.608333,6.593939,6.558207,6.609217,6.576768,6.406818,6.383207,6.608712,6.495328,6.534596,6.124874,6.353157,5.907197,6.034091,5.990657,5.925884,6.378662,6.375884,4.105177,2.735227,3.010227,4.830303,4.15202,4.874242,6.052273,6.318939,6.355934,4.503788,2.834722,3.581944,4.267803,4.135732,4.878409,5.899874,6.114646,6.403157,5.892045,6.179924,6.210985,4.224242,4.205682,4.467803


22
9.0
Max total significant accuracy then averaged 0.7653900112233446 
-dataGCV-BCcn3
sigmoid
C: 2.5 


In [136]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
